In [1]:
import pandas as pd
import numpy as np
import json
from google.cloud import bigquery

In [2]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [3]:
#Nombre de lignes totales dans le JSON : 1320229

In [4]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.event_properties"

In [5]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [6]:
# Observation de la colonne Games, on a des listes de jeu. Sur le chunk de 100K on a 158 combinaisons différentes
#tmp = pd.read_csv('../raw_data/event_properties.csv')
#tmp['Key_Type'].value_counts()

In [7]:
tmp.columns

NameError: name 'tmp' is not defined

In [8]:
columns_rename = {'[Game] Code':'Game_Code','[Profile] Age (days)':'Profile_Age_days','[Scene] Name':'Scene_Name',
                 '[Game] Language':'Game_Language', '[Scene] Previous':'Scene_Previous', '[Time] Spent':'Time_Spent',
                  '[Scene] Next':'Scene_Next', '[Action] Element Type':'Action_Element_Type',
                  '[Scene] Section':'Scene_Section', '[Action] Element Name':'Action_Element_Name',
                 '[Activity] Name':'Activity_Name', '[Scaffolding] Level':'Scaffolding_Level',
                  '[Game] Piece Code':'Game_Piece_Code','[Activity] Nb Wrong Answer':'Activity_Nb_Wrong_Answer',
                  '[Activity] Solved':'Activity_Solved','[Activity] Piece Stamped':'Activity_Piece_Stamped',
                  '[Activity] Piece Asked':'Activity_Piece_Asked','[Activity] Modality':'Activity_Modality',
                  '[Time] Slot':'Time_Slot', '[Error] Type':'Error_Type',
                  '[Activation] Game Code':'Activation_Game_Code',
                  '[Activation] Product Code':'Activation_Product_Code',
                 '[Activation] Piece Code':'Activation_Piece_Code', '[Popup] Name':'Popup_Name',
                  '[Renewal] Type':'Renewal_Type','[Key] Type':'Key_Type','[Toast] Name':'Toast_Name'}

In [12]:
CHUNKSIZE = 100000

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["event_properties"].to_list()).copy()
    ep['event_id']=c["event_id"].copy()
    ep['event_type']=c["event_type"].copy()
    ep['id'] = (i*CHUNKSIZE) + index_init
    ep.rename(columns=columns_rename,inplace=True)
    #ep.to_csv('../raw_data/event_properties.csv',index=False)
    job = client.load_table_from_dataframe(ep, table_id)  # Make an API request. ,job_config=job_config
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

Loaded 100000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 200000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 300000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 400000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 500000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 600000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 700000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 800000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 900000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 1000000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 1100000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 1200000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 1300000 rows and 30 columns to marbotic_dataset.event_properties
Loaded 1320229 rows and 30 columns to marbotic_dataset.event_properties
